# Emojify - Facial Expression to Emoji Mapper

### Libraries

In [1]:
import glob
from IPython.display import Image,HTML, display
import tkinter as tk
from tkinter import * 
import numpy as np
import cv2
import os
import scipy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image,ImageTk
import threading

### Data Splitting for Training and Testing

In [3]:
train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
#training generator for CNN
train_generator = train_datagen.flow_from_directory(
       train_dir,
       target_size=(48,48),
       batch_size=64,
       color_mode="grayscale",
       class_mode='categorical')
#validation generator for CNN
validation_generator = val_datagen.flow_from_directory(
       val_dir,
       target_size=(48,48),
       batch_size=64,
       color_mode="grayscale",
       class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
for i in os.listdir("data/train/"):
    print(str(len(os.listdir("data/train/"+i))) +" "+ i +" images")

3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


In [5]:
for i in os.listdir("data/test/"):
    print(str(len(os.listdir("data/test/"+i))) +" "+ i +" images")

958 angry images
111 disgust images
1024 fear images
1774 happy images
1233 neutral images
1247 sad images
831 surprise images


### Displaying Images/ Data

In [6]:
file_type = "*.jpg" # Assuming all jpg images of folder (Not a single)
sangry = "data/train/angry/"
sdisgust = "data/train/disgust/"
sfear = "data/train/fear/"
shappy = "data/train/happy/"
sneutral = "data/train/neutral/"
ssad = "data/train/sad/"
ssurprise = "data/train/surprise/"
no_of_image_to_show = 5

def display_n_images(sangry,sdisgust,sfear,shappy,sneutral,ssad,ssurprise, file_type, no_of_image_to_show):
    # glob will return list of jpg images
    #  splitting list
    angry = glob.glob(sangry + file_type) 
    angry = angry[0:no_of_image_to_show] 
    disgust = glob.glob(sdisgust + file_type) 
    disgust = disgust[0:no_of_image_to_show]
    fear = glob.glob(sfear + file_type) 
    fear = fear[0:no_of_image_to_show] 
    happy = glob.glob(shappy + file_type) 
    happy = happy[0:no_of_image_to_show]
    neutral = glob.glob(sneutral + file_type) 
    neutral = neutral[0:no_of_image_to_show]
    sad = glob.glob(ssad + file_type) 
    sad = sad[0:no_of_image_to_show]
    surprise = glob.glob(ssurprise + file_type) 
    surprise = surprise[0:no_of_image_to_show]
    
    print("\t\t\t\t    Angry")
    angryd='\t'.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in angry])
    display(HTML(angryd))
    
    print("\n\t\t\t\t    Disgust")
    disgustd='\t'.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in disgust])
    display(HTML(disgustd))
    
    print("\n\t\t\t\t    Fear")
    feard='\t'.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in fear])
    display(HTML(feard))
    
    print("\n\t\t\t\t    Happy")
    happyd='  '.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in happy])
    display(HTML(happyd))
    
    print("\n\t\t\t\t    Neutral")
    neutrald='\t'.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in neutral])
    display(HTML(neutrald))
    
    print("\n\t\t\t\t    Sad")
    sadd='\t'.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in sad])
    display(HTML(sadd))
    
    print("\n\t\t\t\t    Surprise")
    surprised='\t'.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in surprise])
    display(HTML(surprised))
        
display_n_images(sangry,sdisgust,sfear,shappy,sneutral,ssad,ssurprise, file_type, no_of_image_to_show)


				    Angry



				    Disgust



				    Fear



				    Happy



				    Neutral



				    Sad



				    Surprise


### CNN Model using Keras Sequential API

In [7]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))#output=(48-3+0)/1+1=46
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))#output=(46-3+0)/1+1=44
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=devided input by 2 it means 22,22,64
emotion_model.add(Dropout(0.25))#reduce 25% module at a time of output
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape=(48,48,1)))#(22-3+0)/1+1=20
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#10
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))#(10-3+0)/1+1=8
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=4
emotion_model.add(Dropout(0.25))#nothing change
emotion_model.add(Flatten())#here we get multidimension output and pass as linear to the dense so that 4*4*128=2048
emotion_model.add(Dense(1024, activation='relu'))#hddien of 1024 neurons of input 
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))#hddien of 7 neurons of input
plot_model(emotion_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)#save model leyer as model_plot.png
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    

### Training and Testing the Model

In [8]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator( #to fetch the model info from validation generator
       train_generator,
       steps_per_epoch=28709 // 64,
       epochs=50,
       validation_data=validation_generator,
       validation_steps=7178 // 64)

C:\Users\naksh\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\naksh\AppData\Local\Temp\ipykernel_6768\4251697706.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator( #to fetch the model info from validation generator


Epoch 1/50
448/448 [==============================] - 351s 770ms/step - loss: 1.8054 - accuracy: 0.2557 - val_loss: 1.7245 - val_accuracy: 0.3315
Epoch 2/50
448/448 [==============================] - 552s 1s/step - loss: 1.6406 - accuracy: 0.3626 - val_loss: 1.5505 - val_accuracy: 0.4160
Epoch 3/50
448/448 [==============================] - 357s 797ms/step - loss: 1.5292 - accuracy: 0.4113 - val_loss: 1.4740 - val_accuracy: 0.4343
Epoch 4/50
448/448 [==============================] - 331s 739ms/step - loss: 1.4536 - accuracy: 0.4458 - val_loss: 1.3950 - val_accuracy: 0.4707
Epoch 5/50
448/448 [==============================] - 352s 785ms/step - loss: 1.3905 - accuracy: 0.4698 - val_loss: 1.3485 - val_accuracy: 0.4923
Epoch 6/50
448/448 [==============================] - 384s 858ms/step - loss: 1.3376 - accuracy: 0.4930 - val_loss: 1.3003 - val_accuracy: 0.5013
Epoch 7/50
448/448 [==============================] - 344s 767ms/step - loss: 1.2947 - accuracy: 0.5092 - val_loss: 1.2682 - va

### Saving the Model

In [14]:
emotion_model.save_weights('model.h5')

### Output of Emotion Model

In [8]:
cv2.ocl.setUseOpenCL(False)
#emotion dictionary creation
em_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    #bounding box initialization  
    bounding_box = cv2.CascadeClassifier('C:/Users/vikas/AppData/Roaming/Python/Python310/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #to detect the multiple faces and frame them separately   
    n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(crop_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, em_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

1/1 [==============================] - 0s 47ms/step


KeyboardInterrupt: 

### Extracting the Features in Model

In [9]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('model.h5')
cv2.ocl.setUseOpenCL(False)

### Final Ouput (Face Detection using Haarcascade and Mapping of Emoji with the Emotions)

In [ ]:
#  Using openCV haarcascade xml detect the bounding boxes of face in the webcam and predict the emotions:


emotion_dict = {0: "Sad", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Angry", 6: "Surprised"}

emoji_dist={0:"C:/Users/vikas/Desktop/College/ML/Project/emojis/disgust.png",2:"C:/Users/vikas/Desktop/College/ML/Project/emojis/fear.png",3:"C:/Users/vikas/Desktop/College/ML/Project/emojis/happy.png",4:"C:/Users/vikas/Desktop/College/ML/Project/emojis/neutral.png",5:"C:/Users/vikas/Desktop/College/ML/Project/emojis/angry.png",6:"C:/Users/vikas/Desktop/College/ML/Project/emojis/surprised.png"}

global last_frame1
last1_frame1 = np.zeros((448,640,3), dtype = np.uint8)
global cap1
show_text = [0]
global frame_number


#  Using openCV haarcascade xml to detect the bounding boxes of face in the webcam and predict the emotions:

def show_subject():
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("Can't open the camera")
    global frame_number
    length = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number += 1
    if frame_number >= length:
        exit()
    cap1.set(1, frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1,(600,500))
    bounding_box = cv2.CascadeClassifier('C:/Users/vikas/AppData/Roaming/Python/Python310/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x,y,w,h) in num_faces:
        cv2.rectangle(frame1, (x,y-50), (x+w, y+h+10), (255,0,0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1), 0)
        prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        show_text[0] = maxindex
        
    if flag1 is None:
        print("Major error!")
    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1,cv2.COLOR_BGR2RGB)
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image = imgtk)
        root.update()
        lmain.after(10,show_subject)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()

#  Function to display the emoji after detecting the emotion of the face
        
def show_avatar():
    frame2 = cv2.imread(emoji_dist[show_text[0]])
    pic2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(frame2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=emotion_dict[show_text[0]],font=('arial',45,'bold'))
    
    lmain2.configure(image = imgtk2)
    root.update()
    lmain2.after(10,show_avatar())
               
        

if __name__ == '__main__':
    frame_number = 0
    root=tk.Tk()   
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=350)
    
    root.title("Photo To Emoji")            
    root.geometry("1400x900+100+10") 
    root['bg']='black'
    exitButton = Button(root, text='Quit',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    threading.Thread(target = show_subject()).start()
    threading.Thread(target = show_avatar()).start()
    root.mainloop()

1/1 [==============================] - 0s 40ms/step
